In [4]:
from google.transit import gtfs_realtime_pb2
import requests
import pandas as pd
from pathlib import Path

In [5]:
# globals
backend_dir = Path('.').resolve()
output = Path(backend_dir, 'data_out')
target_route = 510

In [6]:
def parse_feed_and_filter_by_route(target_route_id):
  print(target_route_id)
  matching_entities = []
  for entity in feed.entity:
      if entity.HasField("vehicle"):
        vehicle = entity.vehicle
        if vehicle.trip.route_id == str(target_route_id):
            print(vehicle.trip)
            matching_entities.append({
                "id": entity.id,
                "trip_id": vehicle.trip.trip_id,
                "route_id": vehicle.trip.route_id,
                "latitude": vehicle.position.latitude,
                "longitude": vehicle.position.longitude,
                "occupancy_status": vehicle.occupancy_status
            })
  return matching_entities

In [ ]:
# Parse realtime protobuf feed
feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get('https://bustime.ttc.ca/gtfsrt/vehicles')
feed.ParseFromString(response.content)
current_time = pd.Timestamp.now().time()

In [ ]:
# Get all vehicles currently on this route
vehicles_on_route = parse_feed_and_filter_by_route(target_route)

510
trip_id: "194315020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "113478020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "51137020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "138748020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "92963020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "139474020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "33842020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "176031020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "138807020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "155158020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "165012020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "149350020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "92128020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id: "83115020"
schedule_relationship: SCHEDULED
route_id: "510"

trip_id

In [ ]:
# Mapping from dictionary keys to desired column names
key_to_column = {
    "id": "vehicle_id",
    "trip_id": "trip_id",
    "route_id": "route_id",
    "latitude": "vehicle_latitude",
    "longitude": "vehicle_longitude"
}

# Create the DataFrame, remove duplicate vehicles
df = pd.DataFrame(
    [{new_key: d.get(old_key) for old_key, new_key in key_to_column.items()} for d in vehicles_on_route])
df.drop_duplicates()

# Add current timestampt to df
df['now'] = current_time

# Count unique trip_ids
unique_trip_count = df["trip_id"].nunique()
# print(unique_trip_count)

# Save to csv
df.to_csv(Path(output, str(target_route) + "-rt.csv"))

TypeError: unsupported operand type(s) for +: 'PosixPath' and 'str'